# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd62880c040>
├── 'a' --> tensor([[ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703]])
└── 'x' --> <FastTreeValue 0x7fd62880c5b0>
    └── 'c' --> tensor([[ 1.3000,  0.2663,  0.9359,  0.8431],
                        [-0.6869,  2.0142,  1.0004,  0.2988],
                        [ 0.4094, -0.0973, -0.1779,  0.4278]])

In [4]:
t.a

tensor([[ 1.6003,  0.5902,  0.9084],
        [ 0.9958, -0.8620,  0.6703]])

In [5]:
%timeit t.a

63 ns ± 0.0657 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd62880c040>
├── 'a' --> tensor([[-0.4745,  0.6453,  0.1441],
│                   [-0.2797, -1.8246,  0.3933]])
└── 'x' --> <FastTreeValue 0x7fd62880c5b0>
    └── 'c' --> tensor([[ 1.3000,  0.2663,  0.9359,  0.8431],
                        [-0.6869,  2.0142,  1.0004,  0.2988],
                        [ 0.4094, -0.0973, -0.1779,  0.4278]])

In [7]:
%timeit t.a = new_value

73.3 ns ± 0.0635 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703]]),
    x: Batch(
           c: tensor([[ 1.3000,  0.2663,  0.9359,  0.8431],
                      [-0.6869,  2.0142,  1.0004,  0.2988],
                      [ 0.4094, -0.0973, -0.1779,  0.4278]]),
       ),
)

In [10]:
b.a

tensor([[ 1.6003,  0.5902,  0.9084],
        [ 0.9958, -0.8620,  0.6703]])

In [11]:
%timeit b.a

58.4 ns ± 0.0655 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1016,  0.0539,  0.9739],
               [ 2.0233, -0.1016, -0.3686]]),
    x: Batch(
           c: tensor([[ 1.3000,  0.2663,  0.9359,  0.8431],
                      [-0.6869,  2.0142,  1.0004,  0.2988],
                      [ 0.4094, -0.0973, -0.1779,  0.4278]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 25.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 583 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd55dd5aee0>
├── 'a' --> tensor([[[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]],
│           
│                   [[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]],
│           
│                   [[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]],
│           
│                   [[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]],
│           
│                   [[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]],
│           
│                   [[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]],
│           
│                   [[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]],
│           
│                   [[ 1.6003,  0.5902,  0.9084],
│                    [ 0.9958, -0.8620,  0.6703]]])
└── 'x' --> <FastTreeValue 0x7fd5649b5040>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.9 µs ± 76.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd564a56100>
├── 'a' --> tensor([[ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703],
│                   [ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703],
│                   [ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703],
│                   [ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703],
│                   [ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703],
│                   [ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703],
│                   [ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703],
│                   [ 1.6003,  0.5902,  0.9084],
│                   [ 0.9958, -0.8620,  0.6703]])
└── 'x' --> <FastTreeValue 0x7fd55e21be80>
    └── 'c' --> tensor([[ 1.3000,  0.2663,  0.9359,  0.8431],
                        [-0.6869,  2.0142,  1.0004,  0.2988],
                 

In [23]:
%timeit t_cat(trees)

31.7 µs ± 63.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.9 µs ± 55.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]],
       
               [[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]],
       
               [[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]],
       
               [[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]],
       
               [[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]],
       
               [[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]],
       
               [[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]],
       
               [[ 1.6003,  0.5902,  0.9084],
                [ 0.9958, -0.8620,  0.6703]]]),
    x: Batch(
           c: tensor([[[ 1.3000,  0.2663,  0.9359,  0.8431],
                       [-0.6869,  2.0142,  1.0004,  0.2988],
                       [ 0.4094, -0.0973, -0.1779,  0.4278]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703],
               [ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703],
               [ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703],
               [ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703],
               [ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703],
               [ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703],
               [ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703],
               [ 1.6003,  0.5902,  0.9084],
               [ 0.9958, -0.8620,  0.6703]]),
    x: Batch(
           c: tensor([[ 1.3000,  0.2663,  0.9359,  0.8431],
                      [-0.6869,  2.0142,  1.0004,  0.2988],
                      [ 0.4094, -0.0973, -0.1779,  0.4278],
                      [ 1.3000,  0.2663,  0.9359,  0.8431],
                      [-0.6869,  

In [28]:
%timeit Batch.cat(batches)

150 µs ± 333 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

347 µs ± 4.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
